In [1]:
import netCDF4 as nc
import pandas as pd
from matplotlib import pyplot as plt
import xarray as xr
import numpy as np
import scipy
import plotly.express as px
from scipy.stats import mstats
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import math
import csv
import fiona
import geopandas
pd.set_option('display.max_rows', 24000)

In [2]:
#loading in mortality dataset from hierarchy file (available on Zenodo)
#fn_full_agg is the aggregated dataset with full adaptation
fn_full_agg = "/projects/kopp/dtb96/branches/Mortality_Project/data/2_projection/3_impacts/main_specification/raw/single/rcp85/CCSM4/low/SSP3/Agespec_interaction_GMFD_POLY-4_TINV_CYA_NW_w1-oldest-aggregated.nc4"

#fn_inc is the aggregated dataset with income adaptation only
fn_inc_agg = "/projects/kopp/dtb96/branches/Mortality_Project/data/2_projection/3_impacts/main_specification/raw/single/rcp85/CCSM4/low/SSP3/Agespec_interaction_GMFD_POLY-4_TINV_CYA_NW_w1-oldest-incadapt-aggregated.nc4"

#fn_none is the aggregated dataset with no adaptation, only temperature change

fn_none_agg = "/projects/kopp/dtb96/branches/Mortality_Project/data/2_projection/3_impacts/main_specification/raw/single/rcp85/CCSM4/low/SSP3/Agespec_interaction_GMFD_POLY-4_TINV_CYA_NW_w1-oldest-noadapt-aggregated.nc4"

#fn_histclim_agg is the aggregated HistClim dataset
fn_histclim_agg = "/projects/kopp/dtb96/branches/Mortality_Project/data/2_projection/3_impacts/main_specification/raw/single/rcp85/CCSM4/low/SSP3/Agespec_interaction_GMFD_POLY-4_TINV_CYA_NW_w1-oldest-histclim-aggregated.nc4"


In [3]:
#loading in shapefile associated with hierarchy file
#shapefile_5k is the aggregated dataset binned by 5,678 regions
shapefile_5k = "/projects/kopp/dtb96/branches/Mortality_Project/data/impact_high_res"

#shapefile_24k is the full resolution dataset binned by 24,378 regions
shapefile_24k = "/projects/kopp/dtb96/branches/Mortality_Project/data/new_shapefile_nytimes"

#reading in the shapefiles with geopandas
sf_5k = geopandas.read_file(shapefile_5k)
sf_24k = geopandas.read_file(shapefile_24k)


In [4]:
print(sf_5k)

        gadmid           hierid    color  ISO  \
0      28115.0       CAN.1.2.28      1.0  CAN   
1      28116.0     CAN.1.17.403      2.0  CAN   
2      28119.0     CAN.2.34.951      3.0  CAN   
3      28120.0  CAN.11.259.4274      4.0  CAN   
4      28124.0  CAN.11.269.4448      5.0  CAN   
...        ...              ...      ...  ...   
24373   6905.0            BWA.1  24374.0  BWA   
24374   6902.0            BWA.7  24375.0  BWA   
24375   6915.0         BWA.6.16  24376.0  BWA   
24376   6909.0            BWA.5  24377.0  BWA   
24377   6912.0         BWA.4.13  24378.0  BWA   

                                                geometry  
0      MULTIPOLYGON (((-110.05459 53.30730, -110.0911...  
1      POLYGON ((-111.23055 52.91943, -111.23056 52.9...  
2      POLYGON ((-127.68527 55.29570, -127.68806 55.2...  
3      POLYGON ((-77.73080 55.31879, -77.72983 55.318...  
4      POLYGON ((-66.25940 54.99975, -66.24916 54.999...  
...                                                  ... 

In [54]:
#Cleaning raw mortality Datasets so region dimension is helpful. Thanks Brewster!
def fix_region(ds):
    

    #Changes coords/dims/vars so 'region' is region names we can select data by.
    #For e.g. `ds.sel(region='IND.10.121')` should work properly.
  
    return ds.set_coords("regions").swap_dims({"region": "regions"}).rename({"regions": "region"})


ds_full_agg = fix_region(xr.open_dataset(fn_full_agg))
ds_inc_agg = fix_region(xr.open_dataset(fn_inc_agg))
ds_none_agg = fix_region(xr.open_dataset(fn_none_agg))
ds_histclim_agg = fix_region(xr.open_dataset(fn_histclim_agg))

In [55]:
#taking the 20yr rolling quantile values from the 'rebased' dataset
q = (0.17, 0.5, 0.83)
window_n = 20
rebased_qs_full_agg = ds_full_agg["rebased"].rolling({"year": window_n}, center=False).construct("tmp").quantile(q,skipna= False, dim="tmp")
rebased_qs_inc_agg = ds_inc_agg["rebased"].rolling({"year": window_n}, center=False).construct("tmp").quantile(q,skipna= False, dim="tmp")
rebased_qs_none_agg = ds_none_agg["rebased"].rolling({"year": window_n}, center=False).construct("tmp").quantile(q,skipna= False, dim="tmp")
rebased_qs_histclim_agg = ds_histclim_agg["rebased"].rolling({"year": window_n}, center=False).construct("tmp").quantile(q,skipna= False, dim="tmp")



In [56]:
#finding where 83rd %-tile crosses the 17th %-tile, or vice versa
window = slice(2010,2100)

def fun_threshold_full_agg(quant,yr):
    threshold_full_agg = rebased_qs_full_agg.sel(quantile=quant, year=yr)
    return threshold_full_agg

def fun_threshold_inc_agg(quant,yr):
    threshold_inc_agg = rebased_qs_inc_agg.sel(quantile=quant, year=yr)
    return threshold_inc_agg

def fun_threshold_none_agg(quant,yr):
    threshold_none_agg = rebased_qs_none_agg.sel(quantile=quant, year=yr)
    return threshold_none_agg

def fun_threshold_histclim_agg(quant,yr):
    threshold_histclim_agg = rebased_qs_histclim_agg.sel(quantile=quant, year=yr)
    return threshold_histclim_agg

In [57]:
#damages earliest threshold crossing - this gives TOE for the aggregated datasets
#for full adapt
mask_years_over_threshold_damages_full_agg = fun_threshold_full_agg(0.83, 2010) <= fun_threshold_full_agg(0.17, window)
earliest_threshold_year_damages_full_agg = mask_years_over_threshold_damages_full_agg.idxmax(dim="year")

#for income adapt
mask_years_over_threshold_damages_inc_agg = fun_threshold_inc_agg(0.83, 2010) <= fun_threshold_inc_agg(0.17, window)
earliest_threshold_year_damages_inc_agg = mask_years_over_threshold_damages_inc_agg.idxmax(dim="year")

#for no adapt
mask_years_over_threshold_damages_none_agg = fun_threshold_none_agg(0.83, 2010) <= fun_threshold_none_agg(0.17, window)
earliest_threshold_year_damages_none_agg = mask_years_over_threshold_damages_none_agg.idxmax(dim="year")

mask_years_over_threshold_damages_histclim_agg = fun_threshold_histclim_agg(0.83, 2010) <= fun_threshold_histclim_agg(0.17, window)
earliest_threshold_year_damages_histclim_agg = mask_years_over_threshold_damages_histclim_agg.idxmax(dim="year")




In [58]:
#benefits earliest threshold crossing - this gives TOE
#for full adapt
mask_years_over_threshold_benefits_full_agg = fun_threshold_full_agg(0.17, 2010) >= fun_threshold_full_agg(0.83, window)
earliest_threshold_year_benefits_full_agg = mask_years_over_threshold_benefits_full_agg.idxmax(dim="year")

#for income adapt
mask_years_over_threshold_benefits_inc_agg = fun_threshold_inc_agg(0.17, 2010) >= fun_threshold_inc_agg(0.83, window)
earliest_threshold_year_benefits_inc_agg = mask_years_over_threshold_benefits_inc_agg.idxmax(dim="year")

#for no adapt
mask_years_over_threshold_benefits_none = fun_threshold_none(0.17, 2010) >= fun_threshold_none(0.83, window)
earliest_threshold_year_benefits_none = mask_years_over_threshold_benefits_none.idxmax(dim="year")

#for histclim
mask_years_over_threshold_benefits_histclim = fun_threshold_histclim(0.17, 2010) >= fun_threshold_histclim(0.83, window)
earliest_threshold_year_benefits_histclim = mask_years_over_threshold_benefits_histclim.idxmax(dim="year")


In [59]:
#adding TOE of benefits or damages to the shapefiles and removing NaN values
#for full adapt
sf_5k['TOE_benefits_full_agg'] = earliest_threshold_year_benefits_full_agg
sf_5k['TOE_damages_full_agg'] = earliest_threshold_year_damages_full_agg
sf_5k['TOE_benefits_full_agg'] = sf_5k['TOE_benefits_full_agg'].replace(2010,np.NaN)
sf_5k['TOE_damages_full_agg'] = sf_5k['TOE_damages_full_agg'].replace(2010,np.NaN)

#for income adapt
sf_5k['TOE_benefits_inc_agg'] = earliest_threshold_year_benefits_inc_agg
sf_5k['TOE_damages_inc_agg'] = earliest_threshold_year_damages_inc_agg
sf_5k['TOE_benefits_inc_agg'] = sf_24k['TOE_benefits_inc_agg'].replace(2010,np.NaN)
sf_5k['TOE_damages_inc_agg'] = sf_5k['TOE_damages_inc_agg'].replace(2010,np.NaN)

#for no adapt
sf_5k['TOE_benefits_none_agg'] = earliest_threshold_year_benefits_none_agg
sf_5k['TOE_damages_none_agg'] = earliest_threshold_year_damages_none_agg
sf_5k['TOE_benefits_none_agg'] = sf_5k['TOE_benefits_none_agg'].replace(2010,np.NaN)
sf_5k['TOE_damages_none_agg'] = sf_5k['TOE_damages_none_agg'].replace(2010,np.NaN)

#for histclim
sf_5k['TOE_benefits_histclim_agg'] = earliest_threshold_year_benefits_histclim_agg
sf_5k['TOE_damages_histclim_agg'] = earliest_threshold_year_damages_histclim_agg
sf_5k['TOE_benefits_histclim_agg'] = sf_5k['TOE_benefits_histclim_agg'].replace(2010,np.NaN)
sf_5k['TOE_damages_histclim_agg'] = sf_5k['TOE_damages_histclim_agg'].replace(2010,np.NaN)


ValueError: Length of values (5678) does not match length of index (24378)